In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time
import os
import traceback
import geopandas as gpd
from shapely.geometry import Point

# Github/national_address_database/data/state_county/il_crawford.csv.xz

In [2]:
fips_state = pd.read_csv("../data/fips_state.csv", dtype={'fips': str})
fips_county = pd.read_csv("../data/fips_county.csv", dtype={'fips': str})
fips_state.head()

,fips,state,abbr
0,01,alabama,al
1,02,alaska,ak
2,04,arizona,az
3,05,arkansas,ar
4,06,california,ca


In [3]:
fips_county.head()

,fips,names
0,01000,Alabama
1,01001,Autauga County
2,01003,Baldwin County
3,01005,Barbour County
4,01007,Bibb County


In [4]:
fip_il = fips_state.fips[fips_state.abbr == 'il'].iloc[0]
il_county_fips = fips_county[fips_county.fips.str.slice(0,2) == fip_il]
il_county_fips

,fips,names
606,17000,Illinois
607,17001,Adams County
608,17003,Alexander County
609,17005,Bond County
610,17007,Boone County
...,...,...
704,17195,Whiteside County
705,17197,Will County
706,17199,Williamson County
707,17201,Winnebago County


In [5]:
with open('../data/state_county.json', 'r') as file:
    county_file_json = json.load(file)
file.close()

county_file_json = county_file_json[0]['contents']
county_file_df = pd.DataFrame.from_dict(county_file_json)
county_file_df

FileNotFoundError: [Errno 2] No such file or directory: '../data/state_county.json'

In [ ]:
il_files = county_file_df.name[county_file_df.name.str.slice(0,2) == 'il']
il_files

In [ ]:
shape_url = "https://www2.census.gov/geo/tiger/TIGER2020PL/LAYER/TABBLOCK/2020/tl_2020_17_tabblock20.zip"
block_df = gpd.read_file(shape_url)

In [ ]:
block_df['geoid20'] = block_df.STATEFP20 + block_df.COUNTYFP20 + block_df.TRACTCE20 + block_df.BLOCKCE20
block_df

In [ ]:
df = pd.read_csv("../data/state_county/il_vermilion.csv.xz")
df

In [ ]:
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=block_df.crs)
geo_df

In [ ]:
joined_df = block_df.sjoin(geo_df, how='inner',predicate='intersects')
joined_df.loc[:, ['state', 'county', 'zip', 'longitude', 'latitude', 'address', 'id','geoid20']]

In [ ]:
for file_name in il_files:
    df = pd.read_csv("../data/state_county/" + file_name)
    geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=block_df.crs)
    joined_df = block_df.sjoin(geo_df, how='inner',predicate='intersects')
    joined_df.loc[:, ['state', 'county', 'zip', 'longitude', 'latitude', 'address', 'id','geoid20']].to_csv("../data/state_county/" + file_name)

In [ ]:
pd.read_csv("../data/state_county/il_vermilion.csv.xz")

In [6]:
import glob

In [7]:
# Directory path where the files are located
directory_path = '../data/address'

In [8]:
# Using glob to select files starting with "17"
files_starting_with_17 = glob.glob(directory_path + '/17*')

# Printing the list of selected files
for file_path in files_starting_with_17:
    print(file_path)

../data/address/17077.csv.xz
../data/address/17091.csv.xz
../data/address/17017.csv.xz
../data/address/17007.csv.xz
../data/address/17099.csv.xz
../data/address/17159.csv.xz
../data/address/17087.csv.xz
../data/address/17013.csv.xz
../data/address/17015.csv.xz
../data/address/17103.csv.xz
../data/address/17189.csv.xz
../data/address/17157.csv.xz
../data/address/17073.csv.xz
../data/address/17129.csv.xz
../data/address/17147.csv.xz
../data/address/17121.csv.xz
../data/address/17163.csv.xz
../data/address/17005.csv.xz
../data/address/17201.csv.xz
../data/address/17031.csv.xz
../data/address/17169.csv.xz
../data/address/17041.csv.xz
../data/address/17127.csv.xz
../data/address/17151.csv.xz
../data/address/17085.csv.xz
../data/address/17003.csv.xz
../data/address/17179.csv.xz
../data/address/17137.csv.xz
../data/address/17143.csv.xz
../data/address/17023.csv.xz
../data/address/17165.csv.xz
../data/address/17029.csv.xz
../data/address/17141.csv.xz
../data/address/17037.csv.xz
../data/addres

In [12]:
sum(pd.read_csv("../data/address/17183.csv.xz").address.isna())

4939

In [14]:
df = pd.read_csv("../data/address/17183.csv.xz")

In [15]:
df.dropna()

,address,GEOID20,longitude,latitude
890,"8062 il state line road,,il,47928",171830111001012,-87.533221,39.986298
891,"8564 il state line road,,il,47928",171830111001012,-87.532719,39.993829
892,"8160 il state line road,,il,47928",171830111001012,-87.532760,39.987867
3577,"3500 west county road 500 north,,il,47928",171830111001044,-87.532856,39.956394
4637,"7650 north il state line road,,il,47928",171830111001033,-87.533526,39.980466
4638,"7868 il state line road,,il,47928",171830111001033,-87.533883,39.983668
4688,"3500 west county road 250 north,,il,47928",171830111002062,-87.533082,39.923091


In [29]:
empty_df_17 = df[df["address"].isna()]

In [17]:
shape_url = "https://www2.census.gov/geo/tiger/TIGER2020PL/LAYER/TABBLOCK/2020/tl_2020_17_tabblock20.zip"
block_df = gpd.read_file(shape_url)

In [18]:
block_df

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,17,057,953800,1047,170579538001047,Block 1047,G5040,NaN,NaN,NaN,S,5365,0,+40.3948787,-090.1583342,"POLYGON ((-90.15877 40.39479, -90.15876 40.395..."
1,17,169,970200,3023,171699702003023,Block 3023,G5040,NaN,NaN,NaN,S,8585,0,+40.1210215,-090.5598714,"POLYGON ((-90.56039 40.12145, -90.55996 40.121..."
2,17,103,000100,4063,171030001004063,Block 4063,G5040,NaN,NaN,NaN,S,1298159,0,+41.8298424,-088.9511439,"POLYGON ((-88.96107 41.83323, -88.95948 41.833..."
3,17,057,953000,4028,170579530004028,Block 4028,G5040,NaN,NaN,NaN,S,2128215,0,+40.5710452,-090.2872662,"POLYGON ((-90.30324 40.57410, -90.30309 40.574..."
4,17,057,953900,3025,170579539003025,Block 3025,G5040,NaN,NaN,NaN,S,0,16309,+40.2499578,-090.3729903,"POLYGON ((-90.37469 40.24967, -90.37434 40.250..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369973,17,031,827902,3018,170318279023018,Block 3018,G5040,NaN,NaN,NaN,S,7251,0,+41.5804352,-087.5348995,"POLYGON ((-87.53520 41.58110, -87.53485 41.581..."
369974,17,031,826000,2006,170318260002006,Block 2006,G5040,NaN,NaN,NaN,S,10129,0,+41.6182382,-087.5290192,"POLYGON ((-87.52933 41.61915, -87.52896 41.619..."
369975,17,031,825803,2025,170318258032025,Block 2025,G5040,NaN,NaN,NaN,S,7840,0,+41.6179119,-087.5575407,"POLYGON ((-87.55786 41.61859, -87.55766 41.618..."
369976,17,031,520600,1024,170315206001024,Block 1024,G5040,NaN,NaN,NaN,S,4504,0,+41.6916564,-087.5395049,"POLYGON ((-87.54011 41.69150, -87.54011 41.691..."


In [26]:
block_df_17183 = block_df[block_df["COUNTYFP20"]=='183']

In [28]:
block_df_17183

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
547,17,183,000800,1053,171830008001053,Block 1053,G5040,NaN,NaN,NaN,S,4936,0,+40.1318325,-087.6424356,"POLYGON ((-87.64274 40.13226, -87.64237 40.132..."
548,17,183,000200,2056,171830002002056,Block 2056,G5040,NaN,NaN,NaN,S,4759,0,+40.1282784,-087.6130613,"POLYGON ((-87.61370 40.12845, -87.61251 40.128..."
549,17,183,010900,2064,171830109002064,Block 2064,G5040,NaN,NaN,NaN,S,6853286,14951,+40.0667887,-087.8453213,"POLYGON ((-87.86337 40.06484, -87.86331 40.064..."
550,17,183,011000,1011,171830110001011,Block 1011,G5040,NaN,NaN,NaN,S,766744,0,+40.0079305,-087.6542796,"POLYGON ((-87.66604 40.00965, -87.66591 40.009..."
551,17,183,000500,2010,171830005002010,Block 2010,G5040,NaN,NaN,NaN,S,28712,0,+40.1559662,-087.6130347,"POLYGON ((-87.61502 40.15616, -87.61425 40.156..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367945,17,183,010300,3130,171830103003130,Block 3130,G5040,NaN,NaN,NaN,S,3509239,0,+40.3071686,-087.7235139,"POLYGON ((-87.73783 40.31433, -87.73744 40.314..."
367946,17,183,010702,2046,171830107022046,Block 2046,G5040,NaN,NaN,NaN,S,10436,0,+40.0646751,-087.7037422,"POLYGON ((-87.70439 40.06494, -87.70439 40.065..."
367947,17,183,000100,2013,171830001002013,Block 2013,G5040,NaN,NaN,NaN,S,52325,0,+40.1230198,-087.6302178,"POLYGON ((-87.63130 40.12445, -87.63011 40.124..."
367948,17,183,000100,2022,171830001002022,Block 2022,G5040,NaN,NaN,NaN,S,24893,0,+40.1235050,-087.6220812,"POLYGON ((-87.62281 40.12448, -87.62255 40.124..."


In [37]:
empty_df_17

,address,GEOID20,longitude,latitude
0,NaN,171830101002028,NaN,NaN
1,NaN,171830108004017,NaN,NaN
2,NaN,171830108001030,NaN,NaN
3,NaN,171830012001029,NaN,NaN
4,NaN,171830008002043,NaN,NaN
...,...,...,...,...
4941,NaN,171830110001102,NaN,NaN
4942,NaN,171830110001100,NaN,NaN
4943,NaN,171830101003151,NaN,NaN
4944,NaN,171830101003052,NaN,NaN


In [38]:
block_df_17183 = block_df_17183.astype({'GEOID20':'int'})

In [39]:
temp = pd.concat([empty_df_17.GEOID20, block_df_17183.GEOID20]).value_counts()

In [42]:
missing = temp[temp == 1]

In [43]:
missing

171830111001033    1
171830111001012    1
171830111001044    1
171830111002062    1
Name: GEOID20, dtype: int64

In [44]:
empty_df_17[empty_df_17['GEOID20'].isin(missing)]

,address,GEOID20,longitude,latitude


In [52]:
df_adding = pd.DataFrame(missing).reset_index().rename(columns = {'index': 'GEOID20'}).iloc[:, [0]]

In [57]:
result = pd.concat([df_adding, df], axis=0)

In [66]:
result.reset_index(inplace=True)

In [61]:
result.to_csv("")

,GEOID20,address,longitude,latitude
0,171830111001033,NaN,NaN,NaN
1,171830111001012,NaN,NaN,NaN
2,171830111001044,NaN,NaN,NaN
3,171830111002062,NaN,NaN,NaN
0,171830101002028,NaN,NaN,NaN
...,...,...,...,...
4941,171830110001102,NaN,NaN,NaN
4942,171830110001100,NaN,NaN,NaN
4943,171830101003151,NaN,NaN,NaN
4944,171830101003052,NaN,NaN,NaN


In [62]:
df

,address,GEOID20,longitude,latitude
0,NaN,171830101002028,NaN,NaN
1,NaN,171830108004017,NaN,NaN
2,NaN,171830108001030,NaN,NaN
3,NaN,171830012001029,NaN,NaN
4,NaN,171830008002043,NaN,NaN
...,...,...,...,...
4941,NaN,171830110001102,NaN,NaN
4942,NaN,171830110001100,NaN,NaN
4943,NaN,171830101003151,NaN,NaN
4944,NaN,171830101003052,NaN,NaN
